# Devoir Python

Vous devez rendre votre devoir sur GitHub.
Vous avez le droit a tout vos documents et a internet

1. votre depot doit etre privé
2. vous devez inviter comme colaborateur votre chargé de TD/TP
3. Seul le dernier commit avant la fin de la séance sera corrigé.


Ex 1: Integrale de Romberg

Ecrire une fonction integ_romberg(f, a, b, epsilon=1e-6) permettant de calculer l’intégrale numérique de la fonction f entre les bornes a et b avec une précision epsilon selon la méthode de Romberg (https://fr.wikipedia.org/wiki/M%C3%A9thode_de_Romberg).

Il s’agit d’une méthode qui permet d’améliorer les méthodes usuelles de calcul numérique des intégrales, comme la méthode des Trapèzes.
On montre qu’en combinant judicieusement les valeurs obtenues par la méthode des Trapèzes pour différentes subdivisons de l’intervalle d’intégration, on augmente l’ordre de convergence (sa vitesse de convergence). 


In [1]:
import numpy as np
import matplotlib.pyplot as plt

def f1(x):
   f1 = x
   return f1;

def trapezoid(f,a,b,N): #Méthode copié du TP X sur les trapézes pour approximer le calcul numérique d'une intégrale
    h   = (b-a)/N
    xi  = np.linspace(a,b,N+1)
    fi  = f(xi)
    s   = 0.0
    for i in range(1,N):
        s = s + fi[i]
    s = (h/2)*(fi[0] + fi[N]) + h*s
    return s

def romberg(f,a,b,eps,nmax):
    Q         = np.zeros((nmax,nmax),float) #Initialisation 
    converged = 0
    for i in range(0,nmax): #boucle de 0 a la valeur maximal passée en paramétre 
        N      = 2**i
        Q[i,0] = trapezoid(f,a,b,N)
        for k in range(0,i):
            n        = k + 2
            Q[i,k+1] = 1.0/(4**(n-1)-1)*(4**(n-1)*Q[i,k] - Q[i-1,k])
        if (i > 0):
            if (abs(Q[i,k+1] - Q[i,k]) < eps): #Conditions de sorti si on converge
               converged = 1
               break
    print (Q[i,k+1],N,converged)   
    return Q[i,k+1],N,converged


a  = 0.0;b = 2.0 
romberg(f1,a,b,1.0e-6,10)

2.0 2 1


(2.0, 2, 1)

Ex 2: Équation d’état de l’eau à partir de la dynamique moléculaire

Afin de modéliser les planètes de type Jupiter, Saturne, ou même des exo-planètes très massives (dites « super-Jupiters »), la connaissance de l’équation d’état des composants est nécessaire. Ces équations d’état doivent être valables jusqu’à plusieurs centaines de méga-bar ; autrement dit, celles-ci ne sont en aucun cas accessibles expérimentalement. On peut cependant obtenir une équation d’état numériquement à partir d’une dynamique moléculaire.

Le principe est le suivant : on place dans une boite un certain nombre de particules régies par les équations microscopiques (Newton par exemple, ou même par des équations prenant en considération la mécanique quantique) puis on laisse celles-ci évoluer dans la boite ; on calcule à chaque pas de temps l’énergie interne à partir des intéractions électrostatiques et la pression à partir du tenseur des contraintes. On obtient en sortie l’évolution du système pour une densité fixée (par le choix de taille de la boite) et une température fixée (par un algorithme de thermostat que nous ne détaillerons pas ici).

On se propose d’analyser quelques fichiers de sortie de tels calculs pour l’équation d’état de l’eau à très haute pression. Les fichiers de sortie sont disponibles ici; leur nom indique les conditions thermodynamiques correspondant au fichier, p.ex. 6000K_30gcc.out pour T=6000
K et ρ=30 gcc. Le but est, pour chaque condition température-densité, d’extraire l’évolution de l’énergie et de la pression au cours du temps, puis d’en extraire la valeur moyenne ainsi que les fluctuations. Il arrive souvent que l’état initial choisi pour le système ne corresponde pas à son état d’équilibre, et qu’il faille donc « jeter » les quelques pas de temps en début de simulation qui correspondent à cette relaxation du système. Pour savoir combien de temps prend cette relaxation, il sera utile de tracer l’évolution au cours du temps de la pression et l’énergie pour quelques simulations. Une fois l’équation d’état P(ρ,T) et E(ρ,T) extraite, on pourra tracer le réseau d’isothermes.

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as P

# Ouvrir le répertoire 
path = "~/outputs";#Entrez le path de votre répértoire avec les outputs
all_files = os.listdir( path ); #Récupére tout les fichiers ET dossiers il faudrait écrire un filtre pour filtrer les éventuels dossiers présent

# Afficher tout les fichier
for filename in all_files:
  fichier = open(filename,'r');
  print("Calcul moyenne et écart type pour le fichier "+str(filename));
  lines = fichier.readlines();
  # fermez le fichier après avoir lu les lignes
  fichier.close();
  # Itérer sur les lignes
  nbligne=0;
  potentialArray=np.zeros(0);
  kineticArray=np.zeros(0);   #On initialise les tableaux des valeurs physique a 0;
  pressureArray=np.zeros(0);
  for line in lines:
    if (nbligne>21) :
      splitted_line = line.split();
      potientialEnergy = splitted_line[0];
      kineticEnergy = splitted_line[1];                           # on récupérer chaque valeurs physique a partir de la 20iem valeur et ont insére dans les
      pressure = splitted_line[2];                                # tableaux associé
      potentialArray = np.append(potentialArray, potientialEnergy);
      kineticArray = np.append(kineticArray, kineticEnergy);
      pressureArray = np.append(pressureArray, pressure);
    nbligne=nbligne+1;
  
  print("Moyenne des énérgies potentiel:");
  print(np.average(potentialArray));
  print("Moyenne des énérgies kinétiques:");
  print(np.average(kineticArray));
  print("Moyenne des pressions:");
  print(np.average(pressureArray));
  print("Ecart type des énérgies potentiel:");
  print(np.nanstd(potentialArray));
  print("Ecart type des énérgies kinétiques:");
  print(np.nanstd(kineticArray));
  print("Ecart type des pressions:");
  print(np.nanstd(pressureArray));

  if (filename == "6000K_07gcc.out"): #Mettre une condition toujours valide si le nom du fichier n'est pas bon pour avoir les graph
    P.plot(potentialArray, [-686, -685], 'r-', lw=2) # potentiel en rouge
    P.plot(kineticArray, [0.76, 0.77], 'b-', lw=2) # kinetic en bleu
    P.plot(pressureArray, [1460, 1510], 'g-', lw=2) # pression en vert
    P.savefig('StraightLine.png')
    P.show()





        
    
    


Ex 3: Le problème du voyageur de commerce

Le problème du voyageur de commerce est un problème d’optimisation consistant à déterminer le plus court chemin reliant un ensemble de destinations. Il n’existe pas d’algorithme donnant la solution optimale en un temps raisonnable (problème NP-complet), mais l’on peut chercher à déterminer des solutions approchées. On va se placer ici dans le cas d’un livreur devant desservir une seule fois chacune desndestinations d’une ville américaine où les rues sont agencées en réseau carré. 

On utilise la « distance deManhattan »  entre deux points 𝐴(𝑥𝐴,𝑦𝐴) et 𝐵(𝑥𝐵,𝑦𝐵) : 𝑑(𝐴,𝐵) =|𝑥𝐵−𝑥𝐴|+|𝑦𝐵−𝑦𝐴|.

En outre, on se place dans le cas où les coordonnées des destinations sont entières, comprises entre 0 (inclus) et TAILLE = 50 (exclus). Deux destinations peuvent éventuellement avoir les mêmes coordonnées. Les instructions suivantes doivent permettre de définir les classes nécessaires (Ville et Trajet) et de développer un algorithme approché (heuristiques) : l’algorithme du plus proche voisin. 

Seules la librairie standard et la librairie numpy sont utilisables si nécessaire. Implementer les classes et methodes suivante:


Classe Ville:
* __init__(): initialisation d’une ville sans destination.
* aleatoire(n): création de n destinations aléatoires.
* nb_trajet(): retourne le nombre total (entier) de trajets :(𝑛−1)!/2(utilisermath.factorial()).
* distance(i, j): retourne la distance (Manhattan) entre les deux destinations de numéro i et j


Classe Trajet:
* __init__(ville, etapes=None): initialisation sur une ville. Si la liste etapes n’est pas spécifiée, le trajet par défaut est celui suivant les destinations de ville.
* longueur(): retourne la longueur totale du trajetbouclé(i.e. revenant à son point de départ).


Plus proche voisin:
* Ville.plus_proche(i, exclus=[]): retourne la destination la plus proche de la destinationi(au sens de Ville.distance()), hors les destinations de la liste exclus
* Ville.trajet_voisins(depart=0): retourne un Trajet déterminé selon l’heuristique des plus proches voisins (i.e. l’étape suivante est la destination la plus proche hors les destinations déjà visitées) en partant de l’étape initiale depart

Optimisation:
* Proposer un algorithme qui propose une meilleur alternative au "plus proche voisin".

Interface:
* Ville.figure(trajet=None): Afficher le plande la ville et le trajet obtenue (utiliser matplotlib.step()pour des trajets de type « Manhattan »)

In [ ]:
import math
import numpy as N
import pytest

import matplotlib.pyplot as P

class Ville:

    def __init__(self):
        #Initialisation d'une ville sans destination.
        self.destinations = N.array([]).reshape(-1, 2)

    def aleatoire(self, n=20):
        #Création de *n* destinations aléatoires.
        self.destinations = N.random.randint(TAILLE, size=(n, 2));
    def nb_trajets(self):
        #Retourne le nombre total (entier) de trajets: (n-1)!/2.
        ndest = len(self.destinations);
        if ndest > 2:     
            return int(math.factorial(ndest - 1) / 2)
        elif ndest > 0: # on test que ndest >0 et >2 pour éviter le cas entre 0 et 2 ou on a factoriel d'un nombre négatif
            return 1
        else: 
            return 0
    def distance(self, i, j): 
        #Retourne la distance Manhattan-L1 entre les destinations i et j.
        return N.abs(self.destinations[i] - self.destinations[j]).sum();
    def plus_proche(self, i, exclus=[]):
        #Retourne la destination la plus proche de la destination *i*, hors les
        #destinations de la liste `exclus`.
        voisins = [ j for j in range(len(self.destinations))
                    if j != i and j not in exclus ]
        distances = [ self.distance(i, j) for j in voisins ]

        return voisins[N.argmin(distances)]

    def trajet_voisins(self, depart=0):
        #Retourne un `Trajet` déterminé selon l'heuristique des plus proches
        #voisins (i.e. l'étape suivante est la destination la plus proche hors
        #les destinations déjà visitées) en partant de l'étape initiale `depart`.
        ndest = len(self.destinations)
        if depart is None:     # Boucle sur tous les départs possibles
            trajets = [ self.trajet_voisins(depart=i) for i in range(ndest) ]
            longueurs = [ t.longueur() for t in trajets ]

            return trajets[N.argmin(longueurs)]
        else:                  # Départ imposé
            etapes = [depart]
            while len(etapes) < ndest:
                i = etapes[-1]
                j = self.plus_proche(i, exclus=etapes[:-1])
                etapes.append(j)
            return Trajet(self, etapes)
    def optimisation_trajet(self, trajet):
        #Retourne le trajet le plus court de tous les trajets « voisins » à
        #`trajet` (i.e. résultant d'une simple interversion de 2 étapes).
        ndest = len(self.destinations)
        trajets = [ trajet.interversion(i, j)
                    for i in range(ndest) for j in range(i+1, ndest) ]
        longueurs = [ t.longueur() for t in trajets ]
        opt = trajets[N.argmin(longueurs)]
        if opt.longueur() > trajet.longueur():
            opt = trajet
        return opt

    def figure(self, trajet=None, ax=None, offset=0):
        #Visualisation d'une ville et d'un trajet.
        if ax is None:
            fig = P.figure(figsize=(6,6))
            ax = fig.add_subplot(1,1,1, aspect='equal',
                                 xlim=(0, TAILLE), ylim=(0, TAILLE),
                                 title="{} destinations".format(
                                     len(self.destinations)))
            minor_loc = P.matplotlib.ticker.MultipleLocator(1)
            ax.xaxis.set_minor_locator(minor_loc)
            ax.yaxis.set_minor_locator(minor_loc)
            ax.autoscale(False)

        if trajet is None:
            ax.plot(self.destinations[:, 0], self.destinations[:, 1],
                    'ko', zorder=10)
            for i,(x,y) in enumerate(self.destinations):
                #ax.text(x, y, ' '+str(i))
                ax.annotate(str(i), xy=(x, y), xytext=(x+0.5, y+0.5), zorder=10)
        else:
            boucle = N.concatenate((trajet.etapes, [trajet.etapes[0]]))
            ax.step(self.destinations[boucle, 0] + offset,
                    self.destinations[boucle, 1] + offset,
                    label="L={}".format(trajet.longueur()))
        return ax


class Trajet:

     def __init__(self,ville,etapes=None):
        self.ville = ville; #initialise la ville
        if etapes is None:   # si pas étapes passé en paramétre on fixe aux destinations de la villes
            self.etapes = N.arange(len(self.ville.destinations));
        else:
            self.etapes = N.array(etapes);

    def longueur(self):
        #Retourne la longueur totale du trajet *bouclé* (i.e. revenant à son point de départ).
        l = sum( self.ville.distance(self.etapes[i], self.etapes[i+1]);
        for i in range(len(self.etapes)-1) )
          l += self.ville.distance(self.etapes[-1], self.etapes[0]);  
        return l
  

